In [ ]:
import sys
sys.path.append('../')
import cv2
import numpy as np
from PIL import Image
import io
from IPython.display import display, clear_output
from jetbot import Robot, Camera
import time
from servoserial import ServoSerial

# RGB_Lib는 별도의 파일에 저장되어 있어야 합니다.
# 예: 같은 디렉토리에 RGB_Lib.py 라는 이름으로 저장
from RGB_Lib import Programing_RGB

# RGB LED 객체 초기화
# Programing_RGB 클래스 내에서 I2C 버스 1 (0x1b 주소)을 사용하도록 정의되어 있음
RGB = Programing_RGB()

# 로봇 시작 시 초기 LED 상태를 설정 (카메라 준비 중)
try:
    RGB.Set_All_RGB(0x00, 0x00, 0xFF) # 시작 시 파란색 (카메라 대기)
    print("Robot initializing with blue LED...")
except Exception as e:
    print(f"Initial RGB LED setup failed (likely I2C error): {e}")
    print("Please check RGB LED wiring and I2C address/bus settings.")
    
servo_device = ServoSerial() 
servo_device.Servo_serial_control(1, 2048)
time.sleep(0.1)
servo_device.Servo_serial_control(2, 1000)

In [ ]:
# --- [초기 변수 정의] ---
x_position_normalized = 0
y_position_normalized = 0
angle_last = 0
angle = 0

# 로봇 객체 초기화
robot = Robot()

# 카메라 객체 초기화
camera = Camera.instance(width=224, height=224, fps=10)

# 노란색 HSV 범위 정의 (환경에 맞게 조정 필수)
yellow_lower = np.array([20, 100, 100]) # H, S, V
yellow_upper = np.array([30, 255, 255])
blue_lower = np.array([100, 100, 100])
blue_upper = np.array([140, 255, 255])
red_lower_1 = np.array([0, 100, 80])
red_upper_1 = np.array([10, 255, 250])
# 두 번째 빨강 범위 (179도 근처)
red_lower_2 = np.array([160, 100, 20]) # H: 160, S: 100, V: 20
red_upper_2 = np.array([179, 255, 200])# H: 179 (빨강), S: 255, V: 200

# --- [제어 파라미터 (값 조정 필요)] ---
speed_gain_slider_value = 0.55    # 기본 전진 속도 (0.0 ~ 1.0)
steering_gain_slider_value = 0.65   # PD 제어의 비례 이득 (Kp)
steering_dgain_slider_value = 0.13  # PD 제어의 미분 이득 (Kd)
steering_bias_slider_value = 0.0   # 조향 편향 (로봇의 기계적 오차 보정)

# --- [전진 및 정지/감속 로직 파라미터] ---
# 라인이 이미지 Y축 어디에 위치할 때 속도를 조절할지 결정합니다.
# y_position_normalized는 -1 (이미지 상단) ~ 1 (이미지 하단) 범위입니다.
SLOW_DOWN_Y_THRESHOLD = 0.3 # 라인이 이미지 하단 40% 지점 (Y=0.4)에 도달하면 감속 시작
STOP_Y_THRESHOLD = 0.95   # 라인이 이미지 하단 70% 지점 (Y=0.7)에 도달하면 완전히 정지
MIN_SPEED_VALUE = 0.45       # 감속 시 로봇이 유지할 최소 속도 (0.0 ~ 1.0)

# 라인 탐색을 위한 회전 속도 (라인을 찾지 못했을 때)
SEARCH_TURN_SPEED = 0.6 # 라인 미감지 시 제자리 회전 속도 (0.0 ~ 1.0)

def mask_to_color(color,hsv):
    if color == 'yellow':
        final_mask = cv2.inRange(hsv, yellow_lower, yellow_upper)
    elif color == 'blue':
        final_mask = cv2.inRange(hsv, blue_lower, blue_upper)
    elif color == 'red':
        mask1 = cv2.inRange(hsv, red_lower_1, red_upper_1)
        mask2 = cv2.inRange(hsv, red_lower_2, red_upper_2)
        final_mask = cv2.bitwise_or(mask1, mask2)
    return final_mask

print("Starting line tracing loop...")

In [ ]:
# --- [메인 루프 시작] ---
try:
    while True: # 메인 루프는 try 블록 안에 있어야 합니다.
        frame = camera.value # 카메라에서 최신 프레임 가져오기

        # --- [카메라 프레임 유효성 검사] ---
        if frame is None:
            try:
                RGB.Set_All_RGB(0xFF, 0x00, 0x00) # 카메라 에러 시 빨간색 LED
            except:
                pass
            print("경고: 카메라 프레임을 수신할 수 없습니다. 로봇을 정지합니다. 재시도 중...")
            robot.stop()
            print(f"모터 값: 좌 = {robot.left_motor.value:.2f}, 우 = {robot.right_motor.value:.2f} (카메라 오류)")
            time.sleep(0.1)
            continue # 다음 루프에서 다시 시도

        # --- [이미지 전처리] ---
        frame = cv2.resize(frame, (224, 224))
        size_x = frame.shape[1]
        size_y = frame.shape[0]

        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = mask_to_color('yellow',hsv_frame)
        
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        steering_value = 0.0
        current_speed = speed_gain_slider_value # 기본 전진 속도로 초기화
        
        left_speed = 0.0
        right_speed = 0.0

        # --- [라인 감지 로직] ---
        if len(contours) != 0: # 윤곽선(라인)이 하나라도 감지된 경우
            largest_contour = max(contours, key=cv2.contourArea)
            M = cv2.moments(largest_contour)

            # --- [모멘트 계산 및 라인 추적 로직 (내부 try-except)] ---
            try:
                if M["m00"] != 0: # 유효한 모멘트가 계산된 경우
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])

                    cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)

                    x_position_normalized = (cx / size_x) * 2 - 1
                    y_position_normalized = (cy / size_y) * 2 - 1

                    # 조향 이득에 적용될 부스트 팩터 초기화 (기본값 1.0)
                    steering_boost_factor = 1.0 

                    if y_position_normalized > STOP_Y_THRESHOLD:
                        current_speed = 0.0
                        try:
                            RGB.Set_All_RGB(0x00, 0x00, 0xFF) # 파란색 (정지)
                        except:
                            pass
                        print(f"노란색 라인 Y={y_position_normalized:.2f} (>{STOP_Y_THRESHOLD:.2f}). 로봇 정지.")
                    elif y_position_normalized > SLOW_DOWN_Y_THRESHOLD:
                        speed_reduction_factor = (y_position_normalized - SLOW_DOWN_Y_THRESHOLD) / \
                                                 (STOP_Y_THRESHOLD - SLOW_DOWN_Y_THRESHOLD)
                        current_speed = min(MIN_SPEED_VALUE, speed_gain_slider_value * (1 - speed_reduction_factor))
                        steering_boost_factor = 1.0 

                        try:
                            RGB.Set_All_RGB(0xFF, 0xA5, 0x00) # 주황색 (감속 중)
                        except:
                            pass
                        print(f"노란색 라인 Y={y_position_normalized:.2f}. 속도 감소: {current_speed:.2f}. 조향 부스트: {steering_boost_factor:.2f}")
                    else:
                        current_speed = speed_gain_slider_value
                        try:
                            RGB.Set_All_RGB(0x00, 0xFF, 0x00) # 초록색 (주행)
                        except:
                            pass
                        # print(f"노란색 라인 Y={y_position_normalized:.2f}. 최대 속도로 이동 중.")

                    # --- [PID 제어 계산 (조향)] ---
                    angle = x_position_normalized
                    pid = angle * (steering_gain_slider_value * steering_boost_factor) + \
                          (angle - angle_last) * (steering_dgain_slider_value * steering_boost_factor)
                    angle_last = angle
                    steering_value = pid + steering_bias_slider_value
                    
                    # --- [라인 감지 시 모터 값 계산] ---
                    left_speed = float(np.clip(current_speed + steering_value, -1.0, 1.0))
                    right_speed = float(np.clip(current_speed - steering_value, -1.0, 1.0))

            except Exception as e:
                robot.stop()
                try:
                    RGB.Set_All_RGB(0xFF, 0x00, 0xFF) # 보라색 (모멘트 계산 오류 상태 표시)
                except:
                    pass 
                print(f"Error calculating moments for contour: {e}. Stopping robot.")
                # 모터 값 출력: 모멘트 계산 오류 시 모터 정지 상태
                print(f"Motor Values: Left = {robot.left_motor.value:.2f}, Right = {robot.right_motor.value:.2f} (Contour Error)")
                pass
        else: # 윤곽선(노란색 라인)이 전혀 감지되지 않을 때 (라인 미발견)
            try:
                RGB.Set_All_RGB(0xFF, 0x00, 0x00) # 빨간색 (라인 미감지)
            except:
                pass
            current_speed = 0.0
            
            print(f"노란색 라인 미감지. 회전하여 탐색 중. 모터 값: 좌 = {left_speed:.2f}, 우 = {right_speed:.2f}")

        ## 로봇 모터 값 설정
        # current_speed가 0.0이면 정지, 그렇지 않으면 계산된 left_speed, right_speed 적용
        if current_speed < 0.45 and left_speed <0.45 and right_speed <0.45:
            robot.left_motor.value = SEARCH_TURN_SPEED
            robot.right_motor.value = -SEARCH_TURN_SPEED
            print(f"Motor Values: Left = {robot.left_motor.value:.2f}, Right = {robot.right_motor.value:.2f} (Searching)")     
        else:
            robot.left_motor.value = left_speed if left_speed !=0 else 0.45
            robot.right_motor.value = right_speed if right_speed !=0 else 0.45
            # 모터 값 출력: 정상 주행 또는 탐색 회전 상태
            print(f"Motor Values: Left = {robot.left_motor.value:.2f}, Right = {robot.right_motor.value:.2f} (Moving/Searching)")     

        # --- [Jupyter Notebook 출력] ---
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        img_byte_arr = io.BytesIO()
        image.save(img_byte_arr, format='jpeg')
        img_byte_arr = img_byte_arr.getvalue()

        clear_output(wait=True)
        display(Image.open(io.BytesIO(img_byte_arr)))

except KeyboardInterrupt:
    print("\n키보드 인터럽트로 라인 트레이싱이 중지되었습니다.")
    robot.stop()
    try:
        RGB.Set_All_RGB(0x00, 0x00, 0x00) # LED 끔
    except:
        pass
    print(f"모터 값: 좌 = {robot.left_motor.value:.2f}, 우 = {robot.right_motor.value:.2f} (키보드 인터럽트)")
finally:
    if 'camera' in locals() and camera is not None:
        camera.stop()
    robot.stop()
    try:
        RGB.Set_All_RGB(0x00, 0x00, 0x00) # 최종적으로 LED 끔
    except:
        pass
    print("프로그램 종료. 리소스가 해제되었습니다.")
    print(f"최종 모터 값: 좌 = {robot.left_motor.value:.2f}, 우 = {robot.right_motor.value:.2f}")